<a href="https://colab.research.google.com/github/thofes/SJ_distance/blob/main/Yolo_TL_OpenPose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#plots.py und 04_keypoints_from_images updaten

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install


#init Repository - OpenPose
import os
from os.path import exists, join, basename, splitext

#Change the drive to my mounted gdrive
%cd /content

git_repo_url = 'https://github.com/thofes/openpose'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake becaue of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Mounted at /content.gdrive
/content
Cloning into 'yolov5'...
remote: Enumerating objects: 13860, done.
remote: Total 13860 (delta 0), reused 0 (delta 0), pack-reused 13860
Receiving objects: 100% (13860/13860), 25.86 MiB | 27.00 MiB/s, done.
Resolving deltas: 100% (9422/9422), done.
/content/yolov5
     |████████████████████████████████| 1.6 MB 8.0 MB/s 
/content
install new CMake becaue of CUDA10
clone openpose
install system dependencies
Selecting previously unselected package libgflags2.2.
(Reading database ... 123934 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Select

In [ ]:
%cd /content/yolov5
!python detect.py --weights "/content/yolov5/weights_skijumper_only.pt" --save-crop --source '/content.gdrive/MyDrive/Try_1/Neuer Ordner/test' --project '/content.gdrive/MyDrive/Try_1/Neuer Ordner/test' --save-txt

In [ ]:
%cd /content/openpose/build/examples/tutorial_api_python
!python3 04_keypoints_from_images.py --image_dir '/content.gdrive/MyDrive/Try_1/Neuer Ordner/test/exp/crops/skijumper' --keypoint_scale 3 --no_display true --write_images '/content.gdrive/MyDrive/Try_1/Neuer Ordner/test/exp/crops/skijumper' #keypoint_scale 0für nicht normalisiert




In [10]:
#plots.py und 04_keypoints_from_images updaten

#statt 04_keypoints_from_images kann auch keypoints_from_images_TH.py verwendet werden
# Kontrolle beim nächsten Mal

from numpy.core.fromnumeric import shape
import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2
from pathlib import Path



json_path = "/content.gdrive/MyDrive/Manually_Labeled_01042022"

# Liste zum Erstellen des Datensets
list_keypoints_files = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21']


# dictionary to store mapping, labels, and Keypoints
dataset = {
    "labels":[],
    "keypoints":[]
}

PATH = "/content.gdrive/MyDrive/Manually_Labeled_01042022"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass

print(label)

savecounter = 0

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print("Anzahl: " , len(Jumpers))
  

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)
    
#if exp vorhanden or not
      
    Path_check = Path_Pic + '/exp'


    if os.path.isdir(Path_check):

        %cd /content/yolov5
        !python detect.py --weights "/content/yolov5/weights_skijumper_only.pt" --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt
        

        
        folder_dir = Path_Pic + '/exp2/crops/skijumper'
        %cd /content/openpose/build/examples/tutorial_api_python
        #keypoints_from_images_TH.py
        !python3 04_keypoints_from_images.py --image_dir '{folder_dir}' --keypoint_scale 3 --no_display true --write_images '{folder_dir}'  #keypoint_scale 0für nicht normalisiert




        for dec in list_keypoints_files:
 
            pathtofile = Path(Path_Pic + '/exp2/crops/skijumper/keypoints' + dec + '.json')
 
            if pathtofile.is_file():
   
                with open(Path_Pic + '/exp2/crops/skijumper/keypoints' + dec + '.json', "r") as fp:
                    keypoints_json = json.load(fp)

                dataset["keypoints"].append(keypoints_json)
                       
                if int(dec) < 11:
                    dataset["labels"].append(0)
                if int(dec) > 10:
                    dataset["labels"].append(1) 


    else:

        %cd /content/yolov5
        !python detect.py --weights "/content/yolov5/weights_skijumper_only.pt" --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt
        

        
        folder_dir = Path_Pic + '/exp/crops/skijumper'
        %cd /content/openpose/build/examples/tutorial_api_python
        #keypoints_from_images_TH.py
        !python3 04_keypoints_from_images.py --image_dir '{folder_dir}' --keypoint_scale 3 --no_display true --write_images '{folder_dir}'  #keypoint_scale 0für nicht normalisiert




        for dec in list_keypoints_files:
 
            pathtofile = Path(Path_Pic + '/exp/crops/skijumper/keypoints' + dec + '.json')
 
            if pathtofile.is_file():
   
                with open(Path_Pic + '/exp/crops/skijumper/keypoints' + dec + '.json', "r") as fp:
                    keypoints_json = json.load(fp)

                dataset["keypoints"].append(keypoints_json)
                       
                if int(dec) < 11:
                    dataset["labels"].append(0)
                if int(dec) > 10:
                    dataset["labels"].append(1) 

    savecounter += 1

    if savecounter %100 == 0:
        save_json_path = json_path + '/dataset_distance_' + str(savecounter) + '.json'
        with open(save_json_path, "w") as fp:
            json.dump(dataset, fp, indent=4)  




json_path = json_path + '/dataset_distance.json'
with open(json_path, "w") as fp:
  json.dump(dataset, fp, indent=4)  


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/21 /content.gdrive/MyDrive/Manually_Labeled_01042022/NoTelemark/5060/1.jpg: 384x640 1 skijumper, 12.9ms
image 2/21 /content.gdrive/MyDrive/Manually_Labeled_01042022/NoTelemark/5060/10.jpg: 384x640 1 skijumper, 12.9ms
image 3/21 /content.gdrive/MyDrive/Manually_Labeled_01042022/NoTelemark/5060/11.jpg: 384x640 1 skijumper, 8.7ms
image 4/21 /content.gdrive/MyDrive/Manually_Labeled_01042022/NoTelemark/5060/12.jpg: 384x640 1 skijumper, 8.7ms
image 5/21 /content.gdrive/MyDrive/Manually_Labeled_01042022/NoTelemark/5060/13.jpg: 384x640 1 skijumper, 11.1ms
image 6/21 /content.gdrive/MyDrive/Manually_Labeled_01042022/NoTelemark/5060/14.jpg: 384x640 1 skijumper, 8.7ms
image 7/21 /content.gdrive/MyDrive/Manually_Labeled_01042022/NoTelemark/5060/15.jpg: 384x640 1 skijumper, 8.7ms
image 8/21 /content.gdrive/MyDrive/Manually_Labeled_0104

In [ ]:
#Test zum Zuordnen der Bilder

from pathlib import Path

list_keypoints_files = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21']

dataset = {
    "labels":[],
    "keypoints":[]
}
Path_Pic = '/content.gdrive/MyDrive/Try_1/Neuer Ordner/test/'
for dec in list_keypoints_files:
    print("BP1", dec)
    pathtofile = Path(Path_Pic + '/exp/crops/skijumper/keypoints' + dec + '.json')
    print("BP2", pathtofile)
    if pathtofile.is_file():
        print("BP3", int(dec))
        with open(Path_Pic + '/exp/crops/skijumper/keypoints' + dec + '.json', "r") as fp:
            keypoints_json = json.load(fp)

        dataset["keypoints"].append(keypoints_json)
                # wenn ich die Keypoints skaliert auf das gecroppte Bild haben möchte muss ich das hier machen und dann im json File speichern

        if int(dec) < 11:
            dataset["labels"].append(0)
        if int(dec) > 10:
            dataset["labels"].append(1) 
